In [ ]:
import pandas as pd
import numpy as np
!pip install rapidfuzz
from rapidfuzz import process, fuzz

# combine Lead Data with raw propert data

In [ ]:
df = pd.read_csv('clean_df.csv')

In [ ]:
#load property data from other source to 1: attempt to fill null values from REsimpli data,
#2: create onle large dataframe for model

k = pd.read_csv('Kahila County full data.csv')
b = pd.read_csv('Broward County full data.csv')
b2 = pd.read_csv('Broward raw.csv')
p = pd.read_csv('Palm beach raw.csv')
p2 = pd.read_csv('Pasco raw.csv')

#merge the data
kb = pd.concat([k, b, b2, p, p2], ignore_index=True)
kb.shape

(145705, 41)

In [ ]:
kb.columns

Index(['Address', 'Unit #', 'City', 'State', 'Zip', 'County', 'APN',
       'Owner Occupied', 'Owner 1 First Name', 'Owner 1 Last Name',
       'Owner 2 First Name', 'Owner 2 Last Name', 'Mailing Care of Name',
       'Mailing Address', 'Mailing Unit #', 'Mailing City', 'Mailing State',
       'Mailing Zip', 'Mailing County', 'Do Not Mail', 'Property Type',
       'Bedrooms', 'Total Bathrooms', 'Building Sqft', 'Lot Size Sqft',
       'Effective Year Built', 'Total Assessed Value',
       'Last Sale Recording Date', 'Last Sale Amount', 'Total Open Loans',
       'Est. Remaining balance of Open Loans', 'Est. Value',
       'Est. Loan-to-Value', 'Est. Equity', 'MLS Status', 'MLS Date',
       'MLS Amount', 'Lien Amount', 'Marketing Lists', 'Date Added to List',
       'Method of Add'],
      dtype='object')

In [ ]:
df.columns

Index(['First Name', 'Last Name', 'Phone Area Code', 'Phone Number',
       'Email Address', 'Lead Status', 'score', 'Lead Quality', 'Lead Source',
       'Campaign Name', 'Property Street Address', 'Property City',
       'Property State', 'Property Zip', 'Mailing Street Address',
       'Mailing City', 'Mailing State', 'Mailing Zip', 'Bedroom', 'Bathroom',
       'Apporx Sqft', 'Lot Size Sqft', 'Year Buit', 'House Type',
       'Mortgage Amount', 'Tax Assessed Value', 'Tax Billed Amount',
       'Last Sold Price', 'Prior Sale Price', 'Lead Created Date',
       'download_date'],
      dtype='object')

In [ ]:
#ensure tax and last sold are numeric
df['Tax Assessed Value'] =  df['Tax Assessed Value'].str.replace(' ', '') \
                         .str.replace('$', '') \
                         .str.replace(',', '')
df['Tax Assessed Value'] = pd.to_numeric(df['Tax Assessed Value'])

df['Last Sold Price'] =  df['Last Sold Price'].str.replace(' ', '') \
                         .str.replace('$', '') \
                         .str.replace(',', '')
df['Last Sold Price'] = pd.to_numeric(df['Last Sold Price'])

In [ ]:
#prepare to combine lists

#creating necessary columns
kb[['Lead Status', 'Lead Quality', 'Lead Source', 'Campaign Name', 'Lead Created Date', 'download_date', 'Phone Number', 'Phone Area Code']] = 'None'
kb['score'] = -3

#renaming columns
kb.rename(columns={'Mailing Address': 'Mailing Street Address'}, inplace=True)
kb.rename(columns={'Property Type': 'House Type'}, inplace=True)
kb.rename(columns={'Bedrooms': 'Bedroom'}, inplace=True)
kb.rename(columns={'Total Bathrooms': 'Bathroom'}, inplace=True)
kb.rename(columns={'Building Sqft': 'Apporx Sqft'}, inplace=True)
kb.rename(columns={'Lot Size Sqft': 'Lot Size Sqft'}, inplace=True)
kb.rename(columns={'Last Sale Amount' : 'Last Sold Price'}, inplace=True)
kb.rename(columns={'Address' : 'Property Street Address'}, inplace=True)
kb.rename(columns={'City' : 'Property City'}, inplace=True)
kb.rename(columns={'State' : 'Property State'}, inplace=True)
kb.rename(columns={'Zip' : 'Property Zip'}, inplace=True)
kb.rename(columns={'Owner 1 First Name' : 'First Name'}, inplace=True)
kb.rename(columns={'Owner 1 Last Name' : 'Last Name'}, inplace=True)

df.rename(columns={'Year Buit' : 'Effective Year Built'}, inplace=True)

#dropping columns from kb that dont match df and vice versa
dropkb = ['Unit #', 'APN', 'Owner Occupied', 'Owner 2 First Name', 'Owner 2 Last Name', 'Mailing Care of Name',
        'Mailing Unit #', 'Mailing County', 'Do Not Mail',
       'Last Sale Recording Date', 'Total Open Loans',
       'Est. Remaining balance of Open Loans', 'Est. Value',
       'Est. Loan-to-Value', 'Est. Equity', 'MLS Status', 'MLS Date',
       'MLS Amount', 'Lien Amount', 'Marketing Lists','Date Added to List',
       'Method of Add', 'County']

dropdf = ['Email Address', 'Tax Billed Amount', 'Prior Sale Price', 'Mortgage Amount']

kb.drop(dropkb, axis=1, inplace=True)
df.drop(dropdf, axis=1, inplace=True)

In [ ]:
df['Tax Assessed Value'].min()

397.0

In [ ]:
lowest_unique_values = kb['Total Assessed Value'].sort_values().drop_duplicates().head(10)
lowest_unique_values

23046       10
74647     2429
98913     2599
101691    4260
50491     5080
128929    5320
142325    5567
143661    6073
134625    6140
55602     6169
Name: Total Assessed Value, dtype: int64

In [ ]:
kb['Total Assessed Value']

0          67870
1          95650
2         133230
3          30310
4          30390
           ...  
145700    264380
145701    235110
145702    203340
145703     59520
145704    255710
Name: Total Assessed Value, Length: 145705, dtype: int64

In [ ]:
kb.isnull().sum()

Property Street Address       14
Property City                 14
Property State                 0
Property Zip                  18
First Name                 19920
Last Name                     25
Mailing Street Address        35
Mailing City                  76
Mailing State                 49
Mailing Zip                   88
House Type                     0
Bedroom                    39325
Bathroom                    6955
Apporx Sqft                    8
Lot Size Sqft               1163
Effective Year Built         528
Total Assessed Value           0
Last Sold Price             4537
Lead Status                    0
Lead Quality                   0
Lead Source                    0
Campaign Name                  0
Lead Created Date              0
download_date                  0
Phone Number                   0
Phone Area Code                0
score                          0
dtype: int64

In [ ]:
#convert df Total assessed value to approximate Tax assessed value using average Florida laws

homestead_exemption = 50000
soh_cap_adjustment = 0.03

def calculate_tax_assessed_value(row):
    adjusted_value = row['Total Assessed Value'] - homestead_exemption
    taxable_value = adjusted_value * (1 - soh_cap_adjustment)
    return max(taxable_value, 0)

# Apply the function to each row
kb['Tax Assessed Value'] = kb.apply(calculate_tax_assessed_value, axis=1)

kb.drop(columns=['Total Assessed Value'], inplace=True)

In [ ]:
kb['Tax Assessed Value'].min()

0.0

In [ ]:
# Reorder columns
column_order = ['First Name', 'Last Name', 'Phone Area Code', 'Phone Number',
      'Lead Status', 'score', 'Lead Quality', 'Lead Source',
       'Campaign Name', 'Property Street Address', 'Property City',
       'Property State', 'Property Zip', 'Mailing Street Address',
       'Mailing City', 'Mailing State', 'Mailing Zip', 'Bedroom', 'Bathroom',
       'Apporx Sqft', 'Lot Size Sqft', 'Effective Year Built', 'House Type',
       'Tax Assessed Value',
       'Last Sold Price', 'Lead Created Date',
       'download_date']


df = df[column_order]
kb = kb[column_order]

In [ ]:
# See if any of same propertys based on matching property addresses
kb.rename(columns={'Address': 'Property Street Address'}, inplace=True)

# Preprocess columns to lower case and strip whitespace
df['Property Street Address'] = df['Property Street Address'].str.lower().str.strip()
kb['Property Street Address'] = kb['Property Street Address'].str.lower().str.strip()

# Check for matches
matches = kb['Property Street Address'].isin(df['Property Street Address'])
print(matches.sum())

#432 matches

432


In [ ]:
#try to do the same using rapidfuzz

#Function to find matches using rapidfuzz
# def find_matches(df1, df2, column, threshold=90):
#     matches = []
#     for index, value in df1[column].items():
#         if pd.isna(value) or value.strip() == "":
#             continue  # Skip if the value is NaN or an empty string
#         match, score, _ = process.extractOne(value, df2[column].dropna().str.strip(), scorer=fuzz.token_sort_ratio)
#         if score >= threshold:
#             matches.append((value, match, score))
#     return matches

# # Preprocess columns to lower case and strip whitespace
# df['Property Street Address'] = df['Property Street Address'].str.lower().str.strip()
# kb['Property Street Address'] = kb['Property Street Address'].str.lower().str.strip()

# # Find and display matches with a threshold of 95
# matches2 = find_matches(df, kb, 'Property Street Address', threshold=90)

# # Convert matches to a DataFrame
# matches_df2 = pd.DataFrame(matches, columns=['address_df_A', 'address_df_B', 'similarity_score'])

# # Count the number of matches
# num_matches2 = len(matches_df2)

# print(f"Number of matching addresses: {num_matches2}")
# print("Matching addresses:")
# print(matches_df2)

#unfortunatly with a treshhold of 90 and 95, the output was questionsable.


In [ ]:
df['Phone Area Code'].fillna('None', inplace=True)
df['Campaign Name'].fillna('Other', inplace=True)

In [ ]:
# Update specific columns in df with values from kb based on matches, and then combine without overlapping

# Columns to update
update_columns = ['Mailing Street Address', 'Mailing City', 'Mailing State',
       'Mailing Zip', 'House Type', 'Bedroom', 'Bathroom', 'Apporx Sqft',
                     'Lot Size Sqft', 'Last Sold Price', 'Effective Year Built']

# Function to fill and combine
def fill_and_combine(df1, df2, match_column, update_columns):
    # Merge df and kb on the match_column
    merged_df = df.merge(kb, on=match_column, how='left', suffixes=('', '_kb'))

    # Fill missing values in df columns with corresponding values from kb only if df values are empty
    for column in update_columns:
        original_col = merged_df[column]
        update_col = merged_df[f'{column}_kb']

        # Replace only if the original column has empty values
        merged_df[column] = np.where(original_col.isna() | (original_col == ''), update_col, original_col)
        merged_df.drop(columns=[f'{column}_kb'], inplace=True)

    # Extract updated df
    updated_df = merged_df[df.columns]

    # Find unique rows in kb that are not in df
    combined_df = pd.concat([updated_df, kb]).drop_duplicates(subset=[match_column], keep='last').reset_index(drop=True)

    return combined_df


# Apply the function
result_df = fill_and_combine(df, kb, match_column='Property Street Address', update_columns=update_columns)

In [ ]:
#after combined, add a column for are the property and mailing addresses the same
result_df['same_address'] = np.where(
    (result_df['Property Street Address'].isna()) | (result_df['Mailing Street Address'].isna()),
    'unknown',
    np.where(result_df['Property Street Address'] == result_df['Mailing Street Address'], 'yes', 'no')
)

In [ ]:
result_df.shape

(129673, 28)

In [ ]:
result_df.isnull().sum()

First Name                 17511
Last Name                    108
Phone Area Code                0
Phone Number                   0
Lead Status                    0
score                          0
Lead Quality                   0
Lead Source                    0
Campaign Name                  0
Property Street Address        1
Property City                  3
Property State                 4
Property Zip                 108
Mailing Street Address       414
Mailing City                 447
Mailing State                427
Mailing Zip                  461
Bedroom                    38118
Bathroom                    6667
Apporx Sqft                  396
Lot Size Sqft                785
Effective Year Built         651
House Type                   372
Tax Assessed Value           379
Last Sold Price             4552
Lead Created Date              0
download_date                  0
same_address                   0
dtype: int64

In [ ]:
#Renaming improper States
result_df['Property State'].unique()
result_df['Property State'].replace('Boca', 'FL', inplace = True)

In [ ]:
#Renaming Improper Cities
result_df['Property City']. unique()
result_df['Property City'].replace(' DANIA BEACH FL 33004  ', 'Dania Beach', inplace = True)
result_df['Property City'].replace([' Florida 34972', ' FL 33308'], 'Unknown', inplace = True )

In [ ]:
#grouping house types
result_df['House Type'].replace(['SingleFamily','Single family', 'Residential (General) (Single)']
                                , 'Single Family Residential', inplace = True)

result_df['House Type'].replace(['MultiFamily','Multi-Family Dwellings (Generic, 2+)',  ], 'Multi-Unit/Multi-Family Residential', inplace = True)

result_df['House Type'].replace(['Residential Miscellaneous','Residential Common Area (Condo/PUD/etc.)'], 'Residential', inplace = True)

result_df['House Type'].replace('Duplex ', 'Duplex (2 units, any combination)', inplace = True)


In [ ]:
result_df.isnull().sum()

First Name                 17511
Last Name                    108
Phone Area Code                0
Phone Number                   0
Lead Status                    0
score                          0
Lead Quality                   0
Lead Source                    0
Campaign Name                  0
Property Street Address        1
Property City                  3
Property State                 4
Property Zip                 108
Mailing Street Address       414
Mailing City                 447
Mailing State                427
Mailing Zip                  461
Bedroom                    38118
Bathroom                    6667
Apporx Sqft                  396
Lot Size Sqft                785
Effective Year Built         651
House Type                   372
Tax Assessed Value           379
Last Sold Price             4552
Lead Created Date              0
download_date                  0
same_address                   0
dtype: int64

In [ ]:
#making sure numeric values stored numericly
# result_df['Last Sold Price'] = pd.to_numeric(result_df['Last Sold Price'], errors='coerce')
# result_df['Tax Assessed Value'] = pd.to_numeric(result_df['Tax Assessed Value'], errors='coerce')

AttributeError: Can only use .str accessor with string values!

In [ ]:
#save dataframe before handling nulls
result_df.to_csv('clean RE and raw with nulls.csv', index=False)

In [ ]:
#save just RE clean data before handling nulls
#create df of just RE data but clean.

REdf = result_df.drop(result_df[result_df['Lead Status'] == 'None'].index)
REdf.to_csv('clean RE df with nulls.csv', index=False)